In [19]:
import numpy as np
import random
import tsplib95

# Load TSP data from a file
def load_tsp_data(filename):
    problem = tsplib95.load(filename)
    return problem

# Create a random route
def create_route(problem):
    # Adjust for 1-based indexing used in TSPLIB
    return np.random.permutation(range(1, problem.dimension + 1))

# Initialize a population of routes
def initialize_population(pop_size, problem):
    return [create_route(problem) for _ in range(pop_size)]

# Calculate the total distance of a route
def calculate_fitness(route, problem):
    # Wrap around the route to form a complete loop
    wrapped_route = np.append(route, route[0])
    return sum(problem.get_weight(wrapped_route[i], wrapped_route[i + 1]) for i in range(len(route)))

# Selection: Tournament Selection
def tournament_selection(population, problem, k=3):
    selected = []
    for _ in range(len(population)):
        contenders = random.sample(population, k)
        best = min(contenders, key=lambda route: calculate_fitness(route, problem))
        selected.append(best)
    return selected

# Crossover: Ordered Crossover
def ordered_crossover(parent1, parent2):
    size = len(parent1)
    child = [-1] * size
    start, end = sorted(random.sample(range(size), 2))

    # Copy a slice from first parent to child
    for i in range(start, end + 1):
        child[i] = parent1[i]

    # Fill the child with the elements from the second parent in the order they appear
    p2_elements = [item for item in parent2 if item not in child]
    child_idx, p2_idx = 0, 0
    while -1 in child:
        if child[child_idx] == -1:
            child[child_idx] = p2_elements[p2_idx]
            p2_idx += 1
        child_idx += 1

    return child

# Mutation: Swap Mutation
def swap_mutation(route, mutation_rate):
    for i in range(len(route)):
        if random.random() < mutation_rate:
            swap_idx = random.randint(0, len(route) - 1)
            route[i], route[swap_idx] = route[swap_idx], route[i]
    return route
        
def genetic_algorithm(problem, pop_size, generations, mutation_rate):
    population = initialize_population(pop_size, problem)
    for _ in range(generations):
        # Selection
        selected = tournament_selection(population, problem)

        # Crossover
        children = []
        for i in range(0, pop_size, 2):
            if i + 1 < len(selected):
                child1 = ordered_crossover(selected[i], selected[i+1])
                child2 = ordered_crossover(selected[i+1], selected[i])
                children.append(swap_mutation(child1, mutation_rate))
                children.append(swap_mutation(child2, mutation_rate))

        population = children

    # Select the best solution
    best_route = min(population, key=lambda route: calculate_fitness(route, problem))
    return best_route, calculate_fitness(best_route, problem)

# Example execution
problem = load_tsp_data("/Users/mwr/Downloads/NEC_A4/dantzig42.tsp.txt")
best_route, best_distance = genetic_algorithm(problem, pop_size=500, generations=1000, mutation_rate=0.01)
print(f"Best Route: {best_route}\nTotal Distance: {best_distance}")

Best Route: [40, 41, 42, 1, 2, 39, 38, 5, 4, 3, 8, 9, 7, 6, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 26, 27, 24, 25, 10, 12, 11, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13]
Total Distance: 798


In [22]:
import numpy as np
import random
import tsplib95

# Load TSP data from a file
def load_tsp_data(filename):
    problem = tsplib95.load(filename)
    return problem

# Create a random route
def create_route(problem):
    return np.random.permutation(range(1, problem.dimension + 1))

# Initialize a population of routes
def initialize_population(pop_size, problem):
    return [create_route(problem) for _ in range(pop_size)]

# Calculate the total distance of a route
def calculate_fitness(route, problem):
    wrapped_route = np.append(route, route[0])
    return sum(problem.get_weight(wrapped_route[i], wrapped_route[i + 1]) for i in range(len(route)))

# Tournament Selection
def tournament_selection(population, problem, k=3):
    selected = []
    for _ in range(len(population)):
        contenders = random.sample(population, k)
        best = min(contenders, key=lambda route: calculate_fitness(route, problem))
        selected.append(best)
    return selected

# Partially Mapped Crossover (PMX)
def partially_mapped_crossover(parent1, parent2):
    size = len(parent1)
    child = [-1] * size
    start, end = sorted(random.sample(range(size), 2))

    parent1_list = list(parent1)  # Convert parent1 to list for index method
    parent2_list = list(parent2)  # Convert parent2 to list for index method

    # Copy a slice from first parent to child
    for i in range(start, end + 1):
        child[i] = parent1_list[i]

    # Fill the child using elements from the second parent
    for i in range(start, end + 1):
        if parent2_list[i] not in child:
            j, item = i, parent2_list[i]
            while child[j] != -1:
                j = parent1_list.index(child[j])
            child[j] = item

    # Fill remaining positions
    for i in range(size):
        if child[i] == -1:
            child[i] = parent2_list[i]

    return child


# Swap Mutation
def swap_mutation(route, mutation_rate):
    for i in range(len(route)):
        if random.random() < mutation_rate:
            swap_idx = random.randint(0, len(route) - 1)
            route[i], route[swap_idx] = route[swap_idx], route[i]
    return route

# 2-opt Local Search
def two_opt(route, problem):
    improved = True
    while improved:
        improved = False
        for i in range(1, len(route) - 2):
            for j in range(i + 1, len(route)):
                if j - i == 1: continue # Skip adjacent nodes
                new_route = route[:i] + route[j:i-1:-1] + route[j+1:]
                if calculate_fitness(new_route, problem) < calculate_fitness(route, problem):
                    route = new_route
                    improved = True
        if not improved:
            break
    return route

# Elitism
def elitism(population, problem, elite_size=1):
    return sorted(population, key=lambda route: calculate_fitness(route, problem))[:elite_size]

# Dynamic Mutation Rate
def dynamic_mutation_rate(generation, max_generations):
    return max(0.01, 0.1 - (generation / max_generations) * 0.09)

# Genetic Algorithm
def genetic_algorithm(problem, pop_size, generations, initial_mutation_rate):
    population = initialize_population(pop_size, problem)
    best_overall_route = None
    best_overall_distance = float('inf')

    for generation in range(generations):
        mutation_rate = (dynamic_mutation_rate 
                         (generation, generations))
        elite_routes = elitism(population, problem)
        selected = tournament_selection(population, problem)
        children = []
        for i in range(0, len(selected), 2):
            if i + 1 < len(selected):
                child1 = partially_mapped_crossover(selected[i], selected[i+1])
                child2 = partially_mapped_crossover(selected[i+1], selected[i])
                children.append(swap_mutation(child1, mutation_rate))
                children.append(swap_mutation(child2, mutation_rate))

    # Apply local search on children
    children = [two_opt(child, problem) for child in children]

    population = children + elite_routes

    # Update best solution
    for route in population:
        distance = calculate_fitness(route, problem)
        if distance < best_overall_distance:
            best_overall_route = route
            best_overall_distance = distance
            return best_overall_route, best_overall_distance

#Example execution
problem = load_tsp_data("/Users/mwr/Downloads/NEC_A4/dantzig42.tsp.txt")
best_route, best_distance = genetic_algorithm(problem, pop_size=50, generations=100, initial_mutation_rate=0.05)
print(f"Best Route: {best_route}\nTotal Distance: {best_distance}")


KeyboardInterrupt: 

In [ ]:
import numpy as np
import random
import tsplib95

# Load TSP data from a file
def load_tsp_data(filename):
    return tsplib95.load(filename)

# Create a random route
def create_route(problem):
    return np.random.permutation(range(1, problem.dimension + 1))

# Initialize a population of routes
def initialize_population(pop_size, problem):
    return [create_route(problem) for _ in range(pop_size)]

def calculate_fitness(route, problem):
    if route.size == 0:
        return float('inf')

    # Adjust for 1-based indexing, if necessary
    adjusted_route = [city - 1 for city in route] if problem.dimension in route else route
    
    total_distance = 0
    for i in range(len(adjusted_route)):
        start_city = adjusted_route[i]
        end_city = adjusted_route[(i + 1) % len(adjusted_route)]
        if 0 <= start_city < problem.dimension and 0 <= end_city < problem.dimension:
            total_distance += problem.get_weight(start_city, end_city)
    return total_distance



# Tournament Selection
def tournament_selection(population, problem, k=3):
    selected = []
    for _ in range(len(population)):
        contenders = random.sample(population, k)
        best = min(contenders, key=lambda route: calculate_fitness(route, problem))
        selected.append(best)
    return selected

# Roulette Wheel Selection
def roulette_wheel_selection(population, problem):
    fitness_values = [calculate_fitness(route, problem) for route in population]
    max_fitness = max(fitness_values)
    adjusted_fitness = [max_fitness - fitness for fitness in fitness_values]
    total_fitness = sum(adjusted_fitness)
    selection_probs = [f / total_fitness for f in adjusted_fitness]
    selected_indices = np.random.choice(len(population), size=len(population), p=selection_probs)
    return [population[i] for i in selected_indices]

# Partially Mapped Crossover (PMX)
def partially_mapped_crossover(parent1, parent2):
    # Convert parent arrays to lists
    parent1 = list(parent1)
    parent2 = list(parent2)
    size = len(parent1)
    child = [-1] * size
    start, end = sorted(random.sample(range(size), 2))

    for i in range(start, end + 1):
        child[i] = parent1[i]

    for i in range(start, end + 1):
        if parent2[i] not in child:
            j = i
            while parent1[j] in child:
                j = parent1.index(parent2[j]) if parent2[j] in parent1 else j
            child[j] = parent2[i] if j < size else -1

    # Convert child back to a NumPy array before returning
    return np.array(child)


# Order Crossover (OX)
def order_crossover(parent1, parent2):
    size = len(parent1)
    child = [-1] * size
    start, end = sorted(random.sample(range(size), 2))
    child[start:end + 1] = parent1[start:end + 1]
    p2_index = 0
    for i in range(size):
        if child[i] == -1:
            while parent2[p2_index] in child:
                p2_index += 1
            child[i] = parent2[p2_index]
            p2_index += 1
    return child

# Swap Mutation
def swap_mutation(route, mutation_rate):
    mutated_route = route.copy()
    for i in range(len(mutated_route)):
        if random.random() < mutation_rate:
            swap_idx = random.randint(0, len(mutated_route) - 1)
            mutated_route[i], mutated_route[swap_idx] = mutated_route[swap_idx], mutated_route[i]
    return mutated_route

# Inversion Mutation
def inversion_mutation(route, mutation_rate):
    mutated_route = route.copy()
    if random.random() < mutation_rate:
        start, end = sorted(random.sample(range(len(mutated_route)), 2))
        mutated_route[start:end+1] = reversed(mutated_route[start:end+1])
    return mutated_route

# Elitism
def elitism(population, problem, elite_size=1):
    sorted_population = sorted(population, key=lambda route: calculate_fitness(route, problem))
    return sorted_population[:elite_size]

# Dynamic Mutation Rate
def dynamic_mutation_rate(initial_rate, generation, max_generations):
    return initial_rate * (1 - (generation / max_generations))

# Main Genetic Algorithm Function
def genetic_algorithm(problem, pop_size, generations, initial_mutation_rate, elite_size):
    # Ensure population size is even
    if pop_size % 2 != 0:
        pop_size += 1

    population = initialize_population(pop_size, problem)
    best_route = None
    best_distance = float('inf')

    for generation in range(generations):
        current_mutation_rate = dynamic_mutation_rate(initial_mutation_rate, generation, generations)
        selected = tournament_selection(population, problem)
        children = []

        for i in range(0, len(selected) - 1, 2):
            child1 = partially_mapped_crossover(selected[i], selected[i + 1])
            child2 = partially_mapped_crossover(selected[i + 1], selected[i])
            children.extend([child1, child2])
        mutated_children = [swap_mutation(child, current_mutation_rate) for child in children]
        elites = elitism(population, problem, elite_size)
        population = elites + mutated_children
        for route in population:
            distance = calculate_fitness(route, problem)
            if distance < best_distance:
                best_route = route
                best_distance = distance

    return best_route, best_distance

# Execution Block
tsp_file_path = "/Users/mwr/Downloads/NEC_A4/gr17.tsp.txt"  
problem = load_tsp_data(tsp_file_path)

# Define 5 different test cases
test_cases = [
    (50, 100, 0.01, 2),
    (100, 200, 0.02, 3),
    (150, 300, 0.03, 4),
    (200, 400, 0.04, 5),
    (250, 500, 0.05, 1)
]

# Dictionary to store results
test_results = {}

# Run each test case
for test in test_cases:
    pop_size, generations, mutation_rate, elite_size = test
    best_route, best_distance = genetic_algorithm(problem, pop_size, generations, mutation_rate, elite_size)
    test_results[test] = {"route": best_route, "distance": best_distance}

# Saving the results to a file
with open("ga_tsp_test_results.txt", "w") as file:
    for params, result in test_results.items():
        file.write(f"Params: {params}, Best Route: {result['route']}, Total Distance: {result['distance']}\n")

print("All test results saved to 'ga_tsp_test_results.txt'.")